In [ ]:
%reset -f

In [ ]:
import os
os.chdir(r'/home')
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import scipy
import torch.nn as nn
import torch.nn.functional as F
from timeit import default_timer
from torch.utils.data import TensorDataset, DataLoader, random_split

# GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

# Functions
def derivee(y,x):
  return torch.autograd.grad(y,x,torch.ones_like(y),retain_graph=True,create_graph=True)[0]

def som(s):
  return torch.sum(s,dim=-1,keepdim=True)

namedata = 'Struct_4_256'
Data = scipy.io.loadmat(namedata+'.mat')
vox = torch.tensor(Data['vox'], dtype = torch.float32).to(device)

In [ ]:
# NN module
class MLP_gelu(nn.Module):
    def __init__(self,insize,outsize,width,layers):
        super().__init__()
        self.input_layer = nn.Linear(insize,width)
        self.hidden_layers = nn.ModuleList([])
        self.output_layer = nn.Linear(width,outsize)
        for i in range(layers):
          self.linear_layer = nn.Linear(width,width)
          self.hidden_layers.append(self.linear_layer)
        self.layers = layers

    def forward(self, x):
        layer_out = F.gelu(self.input_layer(x))
        for i in range(self.layers):
            layer_out = F.gelu(self.hidden_layers[i](layer_out)) + layer_out
        layer_out = self.output_layer(layer_out)
        return layer_out

# Neural network in PINN
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.NN_trunk = MLP_gelu(insize=6,outsize=3,width=64,layers=4)

    def forward(self, x, y, z, e):
        # Normalisation
        scaler = torch.max(torch.abs(e))
        e = e/scaler
        # NN modulus
        U_tilde = self.NN_trunk(torch.cat([torch.sin(x * 2*torch.pi),
                                     torch.cos(x * 2*torch.pi),
                                     torch.sin(y * 2*torch.pi),
                                     torch.cos(y * 2*torch.pi),
                                     torch.sin(z * 2*torch.pi),
                                     torch.cos(z * 2*torch.pi)], dim=-1))
        ux_tilde, uy_tilde, uz_tilde = U_tilde[...,0:1], U_tilde[...,1:2], U_tilde[...,2:3]
        # Periodic Boundary condition
        ux = (ux_tilde + e[0]*x + e[3]*y + e[4]*z)*scaler
        uy = (uy_tilde + e[3]*x + e[1]*y + e[5]*z)*scaler
        uz = (uz_tilde + e[4]*x + e[5]*y + e[2]*z)*scaler
        return ux, uy, uz

class Homogenisation(nn.Module):
    def __init__(self, struct, epsilon_bar, batch_size=10000):
        super(Homogenisation, self).__init__()

        # Configurations
        self.L = 1.0 # side length of the domain
        self.struct = struct # meta structure
        self.iter = 0
        self.batch_size = batch_size
        # Average Strain
        self.epsilon_bar = epsilon_bar
        # Constitutive Model: hyperelasticity parameters
        self.C10, self.D1 = 1.93, 0.24
        # Solution Space (Displacement)
        self.U = PINN().to(device)
        # Coordinates
        Ndim = len(struct)
        X, Y, Z = torch.meshgrid(torch.linspace(-0.5+0.5/Ndim,0.5-0.5/Ndim,Ndim),
                       torch.linspace(-0.5+0.5/Ndim,0.5-0.5/Ndim,Ndim),
                       torch.linspace(-0.5+0.5/Ndim,0.5-0.5/Ndim,Ndim),
                       indexing='ij')
        L_pde = (self.struct.reshape(-1) == 1).to(device)
        self.x_pde = X.reshape(-1)[:,None].to(device)[L_pde]
        self.y_pde = Y.reshape(-1)[:,None].to(device)[L_pde]
        self.z_pde = Z.reshape(-1)[:,None].to(device)[L_pde]
        self.x_pde.requires_grad = True
        self.y_pde.requires_grad = True
        self.z_pde.requires_grad = True
        self.N = self.x_pde.shape[0]
        # Iteration
        self.iter = 0
        self.scaler = torch.max(torch.abs(epsilon_bar))

    def batch_backward(self):
        energy = 0.0
        for i in range(0, self.N, self.batch_size):
            end_idx = min(i + self.batch_size, self.N)
            x_batch = self.x_pde[i:end_idx]
            y_batch = self.y_pde[i:end_idx]
            z_batch = self.z_pde[i:end_idx]
            batch_energy = self.Psi(x_batch, y_batch, z_batch) / self.N
            batch_energy.backward()  # accumulate gradients
            energy += batch_energy.item()
        return energy

    def Psi(self, x, y, z):  # strain Energy (linear Elasticity) in 3D
        # Displacement
        ux, uy, uz = self.U(x, y, z, self.epsilon_bar)
        # Deformation Gradient
        F = torch.zeros(len(x), 3, 3, dtype=ux.dtype, device=ux.device)
        F[:, 0, 0] = derivee(ux, x).squeeze() + 1.0
        F[:, 1, 1] = derivee(uy, y).squeeze() + 1.0
        F[:, 2, 2] = derivee(uz, z).squeeze() + 1.0
        F[:, 0, 1] = derivee(ux, y).squeeze()
        F[:, 0, 2] = derivee(ux, z).squeeze()
        F[:, 1, 0] = derivee(uy, x).squeeze()
        F[:, 1, 2] = derivee(uy, z).squeeze()
        F[:, 2, 0] = derivee(uz, x).squeeze()
        F[:, 2, 1] = derivee(uz, y).squeeze()
        # Jacobian determinant
        J = torch.linalg.det(F)
        J = J
        # Left Cauchy-Green tensor B = F * F^T
        B = F @ F.transpose(-1, -2)
        I1 = B.diagonal(dim1=-2, dim2=-1).sum(-1)
        Jm23 = J.pow(2.0).pow(-1.0/3.0)
        I1_bar = Jm23 * I1
        # Neo-Hookean deformation density function
        psi = self.C10 * (I1_bar - 3.0) + (1.0 / self.D1) * (J - 1.0).pow(2)
        # scaling
        psi = psi / self.scaler**2
        return psi.sum()

    def Solution(self): # Calculate stress field (first Piola stress) with batching
        U_list = []
        P_list = []
        F_list = []
        for i in range(0, self.N, self.batch_size):
            end_idx = min(i + self.batch_size, self.N)
            x_batch = self.x_pde[i:end_idx]
            y_batch = self.y_pde[i:end_idx]
            z_batch = self.z_pde[i:end_idx]
            # Displacement
            ux, uy, uz = self.U(x_batch, y_batch, z_batch, self.epsilon_bar)
            N_batch = ux.numel()
            F_batch = torch.zeros(N_batch, 3, 3, dtype=ux.dtype, device=ux.device)
            # Deformation gradient F = I + grad(u)
            F_batch[:, 0, 0] = derivee(ux, x_batch).squeeze() + 1.0
            F_batch[:, 1, 1] = derivee(uy, y_batch).squeeze() + 1.0
            F_batch[:, 2, 2] = derivee(uz, z_batch).squeeze() + 1.0
            F_batch[:, 0, 1] = derivee(ux, y_batch).squeeze()
            F_batch[:, 0, 2] = derivee(ux, z_batch).squeeze()
            F_batch[:, 1, 0] = derivee(uy, x_batch).squeeze()
            F_batch[:, 1, 2] = derivee(uy, z_batch).squeeze()
            F_batch[:, 2, 0] = derivee(uz, x_batch).squeeze()
            F_batch[:, 2, 1] = derivee(uz, y_batch).squeeze()
            # Determinant and inverse transpose
            J = torch.linalg.det(F_batch).clamp(min=1e-8)
            F_inv_T = torch.linalg.inv(F_batch).transpose(-1, -2)
            # Invariants
            B = F_batch @ F_batch.transpose(-1, -2)
            I1 = B.diagonal(dim1=-2, dim2=-1).sum(-1)
            Jm23 = J.pow(-2.0 / 3.0)
            I1_bar = Jm23 * I1
            # First-Piola stress
            Jm23_ = Jm23.view(-1, 1, 1)
            I1b_ = I1_bar.view(-1, 1, 1)
            iso_term = 2.0 * self.C10 * (Jm23_ * F_batch - (1.0/3.0) * I1b_ * F_inv_T)
            vol_coeff = ((J - 1.0) * J * 2.0 / self.D1).view(-1, 1, 1)
            vol_term = vol_coeff * F_inv_T
            P_batch = iso_term + vol_term
            # Assemble
            U_batch = torch.cat((ux, uy, uz), dim=-1)
            U_list.append(U_batch)
            P_list.append(P_batch)
            F_list.append(F_batch)
        U = torch.cat(U_list, dim=0)
        U = U - torch.mean(U, dim=0, keepdim=True)
        P = torch.cat(P_list, dim=0)
        F = torch.cat(F_list, dim=0)
        return U, P, F

In [ ]:
E_bar = torch.tensor([0.0,0.0,-0.25,0.0,0.0,0.0]).to(device)
model = Homogenisation(vox,E_bar,batch_size=1000000)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

for ep in range(2000):
    model.train()
    optimizer.zero_grad()
    Loss = model.batch_backward()
    optimizer.step()
    if (ep+2) % 100 == 1:
        print(f'Epoch:{ep+1}, PDE Loss:{Loss:.16e}')


In [ ]:
model.U.eval()
U, Epsilon, Sigma = model.Solution()

ux, uy, uz = U[:, 0:1], U[:, 1:2], U[:, 2:3]

x = (model.x_pde + ux).detach().cpu().numpy()
y = (model.y_pde + uy).detach().cpu().numpy()
z = (model.z_pde + uz).detach().cpu().numpy()
ux = ux.detach().cpu().numpy()
uy = uy.detach().cpu().numpy()
uz = uz.detach().cpu().numpy()

fig = plt.figure(figsize=(8.5, 3))
ax1 = fig.add_subplot(131, projection='3d')
ax2 = fig.add_subplot(132, projection='3d')
ax3 = fig.add_subplot(133, projection='3d')
ax1.set_title('u_x'); ax1.axis('off')
ax2.set_title('u_y'); ax2.axis('off')
ax3.set_title('u_z'); ax3.axis('off')
sc1 = ax1.scatter(x, y, z, c=ux, cmap='jet', s=0.01)
sc2 = ax2.scatter(x, y, z, c=uy, cmap='jet', s=0.01)
sc3 = ax3.scatter(x, y, z, c=uz, cmap='jet', s=0.01)
plt.colorbar(sc1, ax=ax1, shrink=0.5, pad=0.1)
plt.colorbar(sc2, ax=ax2, shrink=0.5, pad=0.1)
plt.colorbar(sc3, ax=ax3, shrink=0.5, pad=0.1)
plt.tight_layout()
plt.show()